In [ ]:
import numpy as np
import logging
import codecs

from pathlib import Path
from operator import add

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
#tf.sysconfig.get_build_info()


from model import create_model
from reader import get_vocab
from argParseDummy import args
from useABAE import loadModel, applyABAE, loadAspectWords, loadInfAspectsDict

args()

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
def getGuiMethod():
    import os,sys,inspect
    currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
    parentdir = os.path.dirname(currentdir) 
    sys.path.insert(0,parentdir) 
    from GuiAskUser import callEvalABAE
    return callEvalABAE #lambda x,y:[0]*len(y)

def evalAllModels(sentences_path, modelNames_list, apaNonApa = "APA"):
    models = []
    infDicts = []
    embDict = {"cord":args.cordEmb_path, "extr":args.extrEmb_path, "extO":args.extOthEmb_path}
    vocabPathDict = {"cord":args.cordVocabDict_path, "extr":args.extrVocabDict_path, "extO":args.extOthVocabDict_path}
    vocabDict = {}
    for key in vocabPathDict:
        vocabDict[key]=get_vocab(vocab_path = vocabPathDict[key])
    for modelName in modelNames_list:
        print("loading: "+modelName)
        infDicts.append(loadInfAspectsDict(args, modelName)) 
        models.append(loadModel(args, vocabDict[modelName[:4]], embDict[modelName[:4]], modelName, int(modelName[-2:])))
    print("finished loading models and inferred aspects")
    
    assert len(models) == len(modelNames_list)

    correctAnswers = [0]*len(models)
    
    evalSentences = codecs.open(sentences_path, 'r', 'utf-8')
    gui = getGuiMethod()
    j = 0
    for sen in evalSentences:
        if sen == " \n" or sen == " \r\n": continue
        aspectList = []
        for i,name in enumerate(modelNames_list):
            pt = applyABAE(sen, models[i], vocabDict[name[:4]])
            aspectNumber = np.argsort(pt)[-1]
            aspectName = infDicts[i][aspectNumber]
            aspectList.append(aspectName)

        res = gui(sen, aspectList)
        
        correctAnswers = list(map(add, correctAnswers, res))
        
        for i,name in enumerate(modelNames_list):
            evalLogPath = args.out_dir_path  / name / ("evalLog"+apaNonApa)
            tmpFile = codecs.open(evalLogPath, "a", "utf-8")
            tmpFile.write(str(j)+": "+ str(res[i]) + "\n")
            tmpFile.close()
        j+=1
    
    resultFile_path = args.out_dir_path / ("evalResults"+apaNonApa)
    resultFile = codecs.open(resultFile_path, "w", "utf-8")
    for i,modelName in enumerate(modelNames_list):
        resultFile.write(modelName+": "+str(correctAnswers[i])+" \n")
    resultFile.close()
        
    return correctAnswers
        



In [ ]:
#eval 100 apa sentences on 15 models
evalAllModels(args.evalSenApa_path, ["cordModelExtOth15", "cordModelExtOth30", "cordModelExtOth60",
                                  "cordModelExtr15", "cordModelExtr30", "cordModelExtr60", 
                                  "extOthModelExtOth15", "extOthModelExtOth30", "extOthModelExtOth60", 
                                  "extOthModelExtr15", "extOthModelExtr30", "extOthModelExtr60", 
                                  "extrModelExtr15", "extrModelExtr30", "extrModelExtr60"], 
                                    apaNonApa = "APA")

In [ ]:
#eval 100 non_apa sentences on 15 models
evalAllModels(args.evalSenNonApa_path, ["cordModelExtOth15", "cordModelExtOth30", "cordModelExtOth60",
                                  "cordModelExtr15", "cordModelExtr30", "cordModelExtr60", 
                                  "extOthModelExtOth15", "extOthModelExtOth30", "extOthModelExtOth60", 
                                  "extOthModelExtr15", "extOthModelExtr30", "extOthModelExtr60", 
                                  "extrModelExtr15", "extrModelExtr30", "extrModelExtr60"], 
                                    apaNonApa = "NonAPA")